# The Movie DataBase (TMDB)
## Foundations Of Computer Science Exam Project
### Stefano Daraio

In [1]:
import pandas as pd
import re
import numpy as np
import json
pd.options.display.float_format = "{:.2f}".format

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head(3) 

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.20,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.38,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.00,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.30,4466


In [3]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


### 1) For each movie, compute the number of cast members

In [4]:
def cast_n(data):
    item_dict = json.loads(data)
    return(len(item_dict))
credits['cast_n'] = credits['cast'].apply(cast_n)
credits[['movie_id','title','cast_n']].head(10)

,movie_id,title,cast_n
0,19995,Avatar,83
1,285,Pirates of the Caribbean: At World's End,34
2,206647,Spectre,83
3,49026,The Dark Knight Rises,158
4,49529,John Carter,27
5,559,Spider-Man 3,143
6,38757,Tangled,13
7,99861,Avengers: Age of Ultron,72
8,767,Harry Potter and the Half-Blood Prince,49
9,209112,Batman v Superman: Dawn of Justice,152


### 2. How many movies do not have a homepage?

In [5]:
conteggio = len(movies[movies['homepage'].isnull()])
print('i film che non hanno homepage sono',conteggio)

i film che non hanno homepage sono 3091


### 3. For each year, how many movies do not have a homepage?

In [6]:

# creo variabile anno# creo va 
movies['years'] = pd.to_datetime(movies['release_date'],format="%Y-%m-%d").dt.year
movies['years'] = movies['years'].replace(np.nan, 'Missing')
#selezione film senza homepage
nohomepage = movies[movies['homepage'].isnull()]
nohomepage[['id', 'years']].groupby('years').count().tail().rename(index=str, columns={'id':'count_noHomepage'})

,count_noHomepage
years,
2013.0,127
2014.0,157
2015.0,110
2016.0,31
Missing,1


### 4. Extract the domain of each homepage.

In [7]:
[re.findall('://([\w+\.+\w+]*)/', i) for i in movies['homepage'] if pd.notnull(i)][8]


[['www.avatarmovie.com'],
 ['disney.go.com'],
 ['www.sonypictures.com'],
 ['www.thedarkknightrises.com'],
 ['movies.disney.com'],
 ['www.sonypictures.com'],
 ['disney.go.com'],
 ['marvel.com']]


### 5. Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

#### Create movie_id

In [8]:
movie_id = movies[['id', 'budget', 'homepage', 'original_language', 'original_title', 'overview', 'popularity', 'release_date','years', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count' ]]
movie_id = movie_id.rename(index=str, columns={'id': 'movie_id'})
movie_id.head(3)

,movie_id,budget,homepage,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count
0,19995,237000000,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,2009-12-10,2009.00,2787965087,162.00,Released,Enter the World of Pandora.,Avatar,7.20,11800
1,285,300000000,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,2007-05-19,2007.00,961000000,169.00,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500
2,206647,245000000,http://www.sonypictures.com/movies/spectre/,en,Spectre,A cryptic message from Bond’s past sends him o...,107.38,2015-10-26,2015.00,880674609,148.00,Released,A Plan No One Escapes,Spectre,6.30,4466


#### Create genre_id

In [637]:
genere_id = pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson = pd.read_json(movies['genres'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    genere_id = pd.concat([genere_id, tabjson])
genere_id = genere_id.rename(index=str, columns={'id': 'genre_id'})
genere_id = genere_id.rename(index=str, columns={'name': 'genre_name'})
genere_id = genere_id[['movie_id','genre_id', 'genre_name']].set_index(['movie_id','genre_id', 'genre_name'])
genere_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, 28.0, Action), (19995, 12.0, Adventure), (19995, 14.0, Fantasy), (19995, 878.0, Science Fiction), (285, 12.0, Adventure)]

#### Create keywords_id

In [638]:
keywords_id = pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson = pd.read_json(movies['keywords'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    keywords_id = pd.concat([keywords_id, tabjson])
keywords_id = keywords_id.rename(index=str, columns={'id': 'key_id'})
keywords_id = keywords_id.rename(index=str, columns={'name': 'key_name'})
keywords = keywords_id[['key_id', 'key_name']]
keywords_id = keywords_id[['movie_id', 'key_id', 'key_name']].set_index(['movie_id', 'key_id', 'key_name'])
keywords_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, 1463.0, culture clash), (19995, 2964.0, future), (19995, 3386.0, space war), (19995, 3388.0, space colony), (19995, 3679.0, society)]

In [639]:
prod_companies_id = pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson = pd.read_json(movies['production_companies'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    prod_companies_id = pd.concat([prod_companies_id, tabjson])
prod_companies_id = prod_companies_id.rename(index=str, columns={'id': 'company_id'})
prod_companies_id = prod_companies_id.rename(index=str, columns={'name': 'company_name'})
prod_companies = prod_companies_id[['company_id', 'company_name']]
prod_companies_id = prod_companies_id[['movie_id', 'company_id', 'company_name']].set_index(['movie_id', 'company_id', 'company_name'])
prod_companies_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, 289.0, Ingenious Film Partners), (19995, 306.0, Twentieth Century Fox Film Corporation), (19995, 444.0, Dune Entertainment), (19995, 574.0, Lightstorm Entertainment), (285, 2.0, Walt Disney Pictures)]

In [640]:
prod_countries_id = pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson = pd.read_json(movies['production_countries'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    prod_countries_id = pd.concat([prod_countries_id, tabjson])
prod_countries_id = prod_countries_id.rename(index=str, columns={'iso_3166_1': 'country_id'})
prod_countries_id = prod_countries_id.rename(index=str, columns={'name': 'country_name'})
prod_countries = prod_countries_id[['country_id', 'country_name']]
prod_countries_id = prod_countries_id[['movie_id', 'country_id', 'country_name']].set_index(['movie_id', 'country_id', 'country_name'])
prod_countries_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, US, United States of America), (19995, GB, United Kingdom), (285, US, United States of America), (206647, GB, United Kingdom), (206647, US, United States of America)]

In [641]:
spoken_languages_id = pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson = pd.read_json(movies['spoken_languages'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    spoken_languages_id = pd.concat([spoken_languages_id, tabjson])
spoken_languages_id = spoken_languages_id.rename(index=str, columns={'iso_639_1': 'language_id'})
spoken_languages_id = spoken_languages_id.rename(index=str, columns={'name': 'language_name'})
spoken_languages = spoken_languages_id[['language_id', 'language_name']]
spoken_languages_id = spoken_languages_id[['movie_id', 'language_id', 'language_name']].set_index(['movie_id', 'language_id', 'language_name'])
spoken_languages_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, en, English), (19995, es, Español), (285, en, English), (206647, fr, Français), (206647, en, English)]

In [642]:
cast = pd.DataFrame()
for film in list(range(0,len(credits['cast']))):
    tabjson = pd.read_json(credits['cast'][film])
    idfilm = credits['movie_id'][film]
    tabjson['movie_id'] = idfilm
    cast = pd.concat([cast, tabjson])
cast = cast.rename(index=str, columns={'id': 'pers_id'})
cast_id = cast[['movie_id', 'cast_id', 'pers_id', 'order', 'credit_id', 'character']]
cast_id = cast_id.set_index(['credit_id'])
cast_id.head(5)

,movie_id,cast_id,pers_id,order,character
credit_id,,,,,
5602a8a7c3a3685532001c9a,19995,242.00,65731.00,0.00,Jake Sully
52fe48009251416c750ac9cb,19995,3.00,8691.00,1.00,Neytiri
52fe48009251416c750aca39,19995,25.00,10205.00,2.00,Dr. Grace Augustine
52fe48009251416c750ac9cf,19995,4.00,32747.00,3.00,Col. Quaritch
52fe48009251416c750ac9d3,19995,5.00,17647.00,4.00,Trudy Chacon


In [644]:
crew = pd.DataFrame()
for film in list(range(0,len(credits['crew']))):
    tabjson = pd.read_json(credits['crew'][film])
    idfilm = credits['movie_id'][film]
    tabjson['movie_id'] = idfilm
    crew = pd.concat([crew, tabjson])
crew = crew.rename(index=str, columns={'id': 'pers_id'})
crew_id = crew[['movie_id', 'credit_id', 'pers_id', 'department', 'job']]
crew_id = crew_id.set_index(['credit_id'])
crew_id.head(5)

,movie_id,pers_id,department,job
credit_id,,,,
52fe48009251416c750aca23,19995,1721.00,Editing,Editor
539c47ecc3a36810e3001f87,19995,496.00,Art,Production Design
54491c89c3a3680fb4001cf7,19995,900.00,Sound,Sound Designer
54491cb70e0a267480001bd0,19995,900.00,Sound,Supervising Sound Editor
539c4a4cc3a36810c9002101,19995,1262.00,Production,Casting


In [645]:
attori_crew = crew[['pers_id', 'gender', 'name']]
attori_cast = cast[['pers_id', 'gender', 'name']]
attori = pd.concat([attori_crew, attori_cast])
attori = attori.sort_values('pers_id').drop_duplicates().set_index('pers_id')
attori.head(5)

,gender,name
pers_id,,
1.00,2.00,George Lucas
2.00,2.00,Mark Hamill
3.00,2.00,Harrison Ford
4.00,1.00,Carrie Fisher
5.00,2.00,Peter Cushing


### 6. For each movie, compute the gross margin (difference between revenue and budget)

In [646]:
movie_id['gross_margin'] = movie_id['revenue'] - movie_id['budget']
movie_id.head(3)

,movie_id,budget,homepage,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count,gross_margin
0,19995,237000000,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,2009-12-10,2009.00,2787965087,162.00,Released,Enter the World of Pandora.,Avatar,7.20,11800,2550965087
1,285,300000000,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,2007-05-19,2007.00,961000000,169.00,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500,661000000
2,206647,245000000,http://www.sonypictures.com/movies/spectre/,en,Spectre,A cryptic message from Bond’s past sends him o...,107.38,2015-10-26,2015.00,880674609,148.00,Released,A Plan No One Escapes,Spectre,6.30,4466,635674609


### 7. For each movie, compute the number of crew members

In [647]:
n_crew = crew[['movie_id','pers_id']].groupby('movie_id').count()
n_crew = members_movie.rename(index=str, columns={'pers_id': 'n_crew'})
n_crew.head(5)

,n_crew
movie_id,
5,88
11,20
12,104
13,93
14,109


### 8. For each movie, compute the number of directors

In [648]:
directors = crew.loc[crew['job'] == 'Director']
num_directors = directors[['movie_id', 'pers_id']].groupby('movie_id').count()
num_directors = pd.merge(num_directors.reset_index(), movie_id.reset_index(), on='movie_id') 
num_directors = num_directors.rename(index=str, columns={'pers_id': 'n_directors'})
num_directors[['movie_id', 'title', 'n_directors']].set_index('movie_id').head(5)

,title,n_directors
movie_id,,
5,Four Rooms,4
11,Star Wars,1
12,Finding Nemo,1
13,Forrest Gump,1
14,American Beauty,1


### 9. For each language, compute the number of movies where such language is spoken.

In [649]:
lang = spoken_languages_id.reset_index().groupby(['language_name','language_id']).count()
lang = lang.iloc[27:]
lang = lang.rename(index=str, columns={'movie_id': 'n_movie'})
lang.sort_values(['n_movie'], ascending=[False]).head(5)

,,n_movie
language_name,language_id,
English,en,4485
Français,fr,437
Español,es,351
Deutsch,de,262
Italiano,it,188


### 10. For each company and each decade, compute the overall revenue

In [650]:
#si calcolano il numero di compagnie per film
pc_id = prod_companies_id.reset_index()
pc_counter = pc_id.groupby('movie_id').count()
pc_counter.head(5)

,company_id,company_name
movie_id,,
5,2,2
11,2,2
12,1,1
13,1,1
14,2,2


In [651]:
#calcolo della decade di ogni film
movie_id['decade'] = pd.to_datetime(movie_id['release_date'], format="%Y-%m-%d").dt.year//10*10
movie_id['decade'] = movie_id.decade.replace(np.nan, 'Missing')
movie_id.sort_values(['release_date'], ascending=[True]).head(3)

,movie_id,budget,homepage,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count,gross_margin,decade
4592,3059,385907,NaN,en,Intolerance,"The story of a poor young woman, separated by ...",3.23,1916-09-04,1916.00,8394751,197.00,Released,The Cruel Hand of Intolerance,Intolerance,7.40,60,8008844,1910.00
4661,3060,245000,NaN,en,The Big Parade,The story of an idle rich boy who joins the US...,0.79,1925-11-05,1925.00,22000000,151.00,Released,NaN,The Big Parade,7.00,21,21755000,1920.00
2638,19,92620000,NaN,de,Metropolis,In a futuristic city sharply divided between t...,32.35,1927-01-10,1927.00,650422,153.00,Released,There can be no understanding between the hand...,Metropolis,8.00,657,-91969578,1920.00


In [652]:
#calcolo le revenue per ogni film e le revenue per ogni film/company
pc = pd.merge(pc_counter.reset_index(), movie_id.reset_index(), on='movie_id')
pc = pc[['movie_id', 'company_id', 'revenue', 'decade']].rename(index=str, columns={'company_id':'count_companies'})
pc['revenue/companies'] = pc['revenue'] / pc['count_companies']
pc.head(3)

,movie_id,count_companies,revenue,decade,revenue/companies
0,5,2,4300000,1990.00,2150000.00
1,11,2,775398007,1970.00,387699003.50
2,12,1,940335536,2000.00,940335536.00


In [653]:
#guadagno per ogni film/compagnia
revenue = pd.merge(pc_id, pc[['movie_id', 'decade', 'revenue', 'count_companies', 'revenue/companies']], on='movie_id')
revenue.head()

,movie_id,company_id,company_name,decade,revenue,count_companies,revenue/companies
0,19995,289.00,Ingenious Film Partners,2000.00,2787965087,4,696991271.75
1,19995,306.00,Twentieth Century Fox Film Corporation,2000.00,2787965087,4,696991271.75
2,19995,444.00,Dune Entertainment,2000.00,2787965087,4,696991271.75
3,19995,574.00,Lightstorm Entertainment,2000.00,2787965087,4,696991271.75
4,285,2.00,Walt Disney Pictures,2000.00,961000000,3,320333333.33


In [654]:
#guadagno totale per ogni compagnia/decade
revenue_grouped = revenue[['company_id','company_name', 'decade', 'revenue/companies']].groupby(['company_id', 'company_name','decade']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
#format(revenue_grouped[['overall_revenue']], '.53g')
revenue_grouped.head(9)

overall_revenue
company_id company_name         decade                 
1.0        Lucasfilm            1970.0     434365670.17
                                1980.0    1187157083.17
                                1990.0     924317558.00
                                2000.0    1892716344.50
                                2010.0      16788459.00
2.0        Walt Disney Pictures 1940.0      83320000.00
                                1990.0    1909229195.00
                                2000.0    4803995711.48
                                2010.0    6003551354.92

### 11. For each decade, compute the company with maximum revenue

In [655]:
revenue_grouped_m = revenue[['company_id', 'decade','revenue/companies']].groupby(['decade', 'company_id']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
revenue_grouped_m = revenue_grouped_m.groupby('decade').max()
revenue_grouped_m = pd.merge(revenue_grouped_m.reset_index(), revenue_grouped.reset_index(), on=['decade', 'overall_revenue'])
revenue_grouped_m.head(20)

,decade,overall_revenue,company_id,company_name
0,1910.0,4197375.50,1307.0,Triangle Film Corporation
1,1910.0,4197375.50,1308.0,Wark Producing Corp.
2,1920.0,26358000.00,8411.0,Metro-Goldwyn-Mayer (MGM)
3,1930.0,200088229.50,1553.0,Selznick International Pictures
4,1930.0,200088229.50,8411.0,Metro-Goldwyn-Mayer (MGM)
5,1940.0,309597150.00,3166.0,Walt Disney Productions
6,1950.0,68500000.00,306.0,Twentieth Century Fox Film Corporation
7,1960.0,307234056.00,7576.0,Eon Productions
8,1970.0,767330337.50,33.0,Universal Pictures
9,1980.0,3082731010.83,4.0,Paramount Pictures


### 12. In each year, how many movies have revenue smaller than the budget?

In [656]:
movies_negative = movie_id.loc[movie_id.gross_margin < 0, ['years','gross_margin']].groupby('years').count().rename(index=str, columns={'gross_margin': 'count_unsuccess'})
movies_negative.tail(10)


,count_unsuccess
years,
2007.0,55
2008.0,64
2009.0,78
2010.0,65
2011.0,70
2012.0,52
2013.0,62
2014.0,59
2015.0,67
